In [2]:
#!/usr/bin/env python
# coding: utf-8

import random
import time
import threading

# Constants for the number of threads and the number of objects to process.
NUM_THREADS = 8
NUM_OBJECTS = 700000

# Global variables to keep track of detected objects and confidence scores.
# These variables are shared across multiple threads and need to be accessed and modified safely.
detected_objects_without_race_handling = 0  # Number of detected objects (without race condition handling)
detected_objects_with_race_handling = 0  # Number of detected objects (with race condition handling)
confidence_score_without_race_handling = 0.0  # Total confidence score (without race condition handling)
confidence_score_with_race_handling = 0.0  # Total confidence score (with race condition handling)
detected_objects = 0  # Total number of detected objects (sequential processing)
confidence_score = 0.0  # Total confidence score (sequential processing)

# A lock object to prevent race conditions when threads access shared resources.
# This lock is used to ensure that only one thread can modify the shared variables at a time.
lock = threading.Lock()

# Arrays to store pre-generated random results for detection and confidence scores.
# These are used to simulate the outcomes of an object detection process.
random_detection_results = []  # Stores random detection results (True/False)
random_confidence_scores = []  # Stores random confidence scores (0.0 - 1.0)


# Function to return the detection result from the pre-generated array.
# Given an index, it returns whether an object was detected at that index.
def detect_object(index):
    return random_detection_results[index]


# Function to return the confidence score from the pre-generated array.
# Given an index, it returns the confidence score associated with the detection at that index.
def compute_confidence_score(index):
    return random_confidence_scores[index]


# Function to detect objects with race condition handling.
# This function is designed to be executed in a threaded environment.
# It uses a lock to safely update shared variables.
def detect_objects_with_race_handling(rank):
    global detected_objects_with_race_handling, confidence_score_with_race_handling
    for i in range(rank, NUM_OBJECTS, NUM_THREADS):
        if detect_object(i):
            # Compute the confidence score for the detected object.
            confidence_score = compute_confidence_score(i)
            with lock:  # Acquire the lock before modifying shared variables.
                detected_objects_with_race_handling += 1
                confidence_score_with_race_handling += confidence_score


# Function to detect objects without race condition handling.
# It updates shared variables without any synchronization mechanism.
# This can lead to a race condition when accessed by multiple threads simultaneously.
def detect_objects_without_race_handling(rank):
    global detected_objects_without_race_handling, confidence_score_without_race_handling
    for i in range(rank, NUM_OBJECTS, NUM_THREADS):
        if detect_object(i):
            confidence_score = compute_confidence_score(i)
            detected_objects_without_race_handling += 1
            confidence_score_without_race_handling += confidence_score


# Function for sequential detection of objects.
# It processes all objects in a single-threaded manner (sequentially).
def detected_objects_seq():
    global detected_objects, confidence_score
    for i in range(NUM_OBJECTS):
        if detect_object(i):
            confidence_score = compute_confidence_score(i)
            detected_objects += 1
            confidence_score += confidence_score


# Main function where the program execution starts.
def main():
    global random_detection_results, random_confidence_scores

    # Seed the random number generator for consistent results.
    random.seed(42)

    # Generate random detection results and confidence scores for simulation purposes.
    random_detection_results = [
        random.randint(0, 1) for _ in range(NUM_OBJECTS)
    ]
    random_confidence_scores = [
        random.uniform(0, 1) for _ in range(NUM_OBJECTS)
    ]

    # Measure execution time for detection with race condition handling.
    start_time = time.time()
    threads_with_race_handling = [
        threading.Thread(target=detect_objects_with_race_handling, args=(i, ))
        for i in range(NUM_THREADS)
    ]
    for thread in threads_with_race_handling:
        thread.start()
    for thread in threads_with_race_handling:
        thread.join()
    end_time = time.time()
    execution_time_with_race_handling = end_time - start_time

    # Output results for detection with race condition handling.
    print("\nDetection results (with race condition handling):")
    print("Total detected objects:", detected_objects_with_race_handling)
    print("Total confidence score:",
          "{:.2f}".format(confidence_score_with_race_handling))
    print("Execution time with race condition handling: {:.2f} seconds".format(
        execution_time_with_race_handling))

    # Measure execution time for detection without race condition handling.
    start_time = time.time()
    threads_without_race_handling = [
        threading.Thread(target=detect_objects_without_race_handling, args=(i, ))
        for i in range(NUM_THREADS)
    ]
    for thread in threads_without_race_handling:
        thread.start()
    for thread in threads_without_race_handling:
        thread.join()
    end_time = time.time()
    execution_time_without_race_handling = end_time - start_time

    # Output results for detection without race condition handling.
    print("\nDetection results (without race condition handling):")
    print("Total detected objects:", detected_objects_without_race_handling)
    print("Total confidence score:",
          "{:.2f}".format(confidence_score_without_race_handling))
    print("Execution time without race condition handling: {:.2f} seconds".
          format(execution_time_without_race_handling))

    # Measure execution time for sequential detection.
    start_time = time.time()
    detected_objects_seq()
    end_time = time.time()
    execution_time_sequential = end_time - start_time

    # Output results for sequential detection.
    print("\nDetection results (sequential):")
    print("Total detected objects:", detected_objects)
    print("Total confidence score:", "{:.2f}".format(confidence_score))
    print("Execution time sequential {:.2f} seconds".format(
        execution_time_sequential))

    # Calculate and print the speedup achieved with race condition handling.
    speedup = execution_time_sequential / execution_time_with_race_handling
    print("Speedup with race condition handling against sequential: {:.2f}".
          format(speedup))
    print("Efficiency of race condition handling version is: {:.2f}".format(
        speedup / NUM_THREADS))

    speedup = execution_time_sequential / execution_time_without_race_handling
    print(
        "Speedup without race condition handling against sequential version: {:.2f}"
        .format(speedup))
    print("Efficiency of without race condition handling version is: {:.2f}".format(
        speedup / NUM_THREADS))


if __name__ == '__main__':
    main()


Detection results (with race condition handling):
Total detected objects: 350371
Total confidence score: 175009.58
Execution time with race condition handling: 0.21 seconds

Detection results (without race condition handling):
Total detected objects: 350371
Total confidence score: 175009.58
Execution time without race condition handling: 0.12 seconds

Detection results (sequential):
Total detected objects: 350371
Total confidence score: 1.55
Execution time sequential 0.09 seconds
Speedup with race condition handling against sequential: 0.44
Efficiency of race condition handling version is: 0.05
Speedup without race condition handling against sequential version: 0.76
Efficiency of without race condition handling version is: 0.10
